### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
school_data_to_load = os.path.join("Resources", "schools_complete.csv")
student_data_to_load = os.path.join("Resources", "students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
# Most analysis is done by creating dataframes and then counting entries
# Count number of unique school entries and student IDs
total_schools = school_data_complete["school_name"].nunique()
total_students = school_data_complete["Student ID"].nunique()

# Budget is calculated by finding unique values, forming them into a dataframe, and then summing them
total_budget_df = school_data_complete["budget"].unique()
total_budget = total_budget_df.sum()

# Average scores are calculated by taking the mean of relevant columns
average_math_score = school_data_complete["math_score"].mean()
average_read_score = school_data_complete["reading_score"].mean()

# Percent passing checks relevant column for a passing score, creates a dataframe of those students, then counts them.
perc_passing_math_df = school_data_complete[school_data_complete["math_score"] >= 70]
perc_passing_math = 100 * perc_passing_math_df["Student ID"].count() / total_students
perc_passing_read_df = school_data_complete[school_data_complete["reading_score"] >= 70]
perc_passing_read = 100 * perc_passing_read_df["Student ID"].count() / total_students

# Merges above dataframes such that only students passing both are included.  These are then counted.
perc_passing_overall_df = pd.merge(perc_passing_math_df, perc_passing_read_df, how = 'inner')
perc_passing_overall = 100 * perc_passing_overall_df["Student ID"].count() / total_students


# Summary table is formed by passing relevant values into dataframe.
district_summary = pd.DataFrame(
    {"Total Schools":[total_schools],
     "Total Students": [total_students],
     "Total Budget": [total_budget],
     "Average Math Score": [average_math_score],
     "Average Reading Score": [average_read_score],
     "% Passing Math": [perc_passing_math],
     "% Passing Reading": [perc_passing_read],
     "% Passing Overall": [perc_passing_overall]
    }
)

# Summary table is formatted for easier reading
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)


# Print summary to screen
district_summary
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [9]:
# Data is grouped by school name and type as these each school is exactly one type. index is false for ease of formatting.
# The mean is used as size and budget will only return the value repeated for each entry and will give average scores.
school_group = school_data_complete.groupby(["school_name","type"], as_index=False)
school_group_df = school_group.mean()


# Earlier dataframes are grouped by school to produce needed data by school.
school_math_group = perc_passing_math_df.groupby("school_name", as_index=False)
school_math_group_df = school_math_group.count()
school_group_df["% Passing Math"] = 100 * school_math_group_df["math_score"] / school_group_df["size"]

school_read_group = perc_passing_read_df.groupby("school_name", as_index=False)
school_read_group_df = school_read_group.count()
school_group_df["% Passing Reading"] = 100 * school_read_group_df["reading_score"] / school_group_df["size"]

school_overall_group = perc_passing_overall_df.groupby("school_name", as_index=False)
school_overall_group_df = school_overall_group.count()
school_group_df["% Passing Overall"] = 100 * school_overall_group_df["reading_score"] / school_group_df["size"]


# Per school budget is total budget divided by number of students.
school_group_df["Per Student Budget"] = school_group_df["budget"] / school_group_df["size"]

# Column order and placement is set.
school_group_reorganized_df = school_group_df[["school_name",
                                               "type",
                                               "size",
                                               "budget",
                                               "Per Student Budget",
                                               "math_score",
                                               "reading_score",
                                               "% Passing Math",
                                               "% Passing Reading",
                                               "% Passing Overall"]]

# All relevant columns are renamed.
school_group_renamed_df = school_group_reorganized_df.rename(columns = {"school_name":"",
                                                                        "type":"School Type",
                                                                        "size":"Total Students",
                                                                        "budget":"Total School Budget",
                                                                        "math_score":"Average Math Score",
                                                                        "reading_score":"Average Reading Score"
})

# Formatting
# School name is made the index of an empty string for formatting
school_group_renamed_df["Total Students"] = school_group_renamed_df["Total Students"].map("{:,.0f}".format)
school_group_renamed_df["Total School Budget"] = school_group_renamed_df["Total School Budget"].map("${:,.2f}".format)
school_group_renamed_df["Per Student Budget"] = school_group_renamed_df["Per Student Budget"].map("${:,.2f}".format)
school_group_renamed_df.set_index("")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [10]:
# Sort above grouped dataframe by best passing.
school_group_best_overall_df = school_group_renamed_df.sort_values("% Passing Overall", ascending=False)
school_group_best_overall_df.set_index("").head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [11]:
# Sort above grouped dataframe by worst passing.
school_group_worst_overall_df = school_group_renamed_df.sort_values("% Passing Overall", ascending=True)
school_group_worst_overall_df.set_index("").head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type